## Swin transformer

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#cd /content/drive/MyDrive/sunandini

In [ ]:
#!unzip "/content/drive/MyDrive/sunandini/trans_data.zip" 

# Installing required packages


In [1]:
!pip install timm
!pip install einops
!pip install tensorboardX
!pip install medpy
!pip install yacs
!pip install torchinfo
!pip install einops

# importing all required packages

In [2]:
# importing all required packages
import math
import copy
import torch.optim as optim
from torch.nn.parallel import DataParallel
from collections import OrderedDict
from torch import nn
from torchinfo import summary
import copy
import torch 
from torch import nn
from torch.nn import init
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import os
from sklearn.metrics import jaccard_score, precision_score, recall_score
import numpy as np
from torch import nn
import torch.utils.checkpoint as checkpoint

from timm.models.layers import DropPath, to_2tuple, trunc_normal_



In [3]:
from model import *

# calling the SwinDeeplab model

In [9]:
model = SwinDeepLab(
    EncoderConfig, 
    ASPPConfig, 
    DecoderConfig
)

C:\Users\rajes\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Printing summary of the model

In [11]:
# summary(model=model,input_size=(25, 3, 224, 224), # (batch_size, color_channels, height, width)
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"])

# defining the conditions for pretrained weights 
## in this code we are not using pretrained weights

In [12]:
# if EncoderConfig.encoder_name == 'swin' and EncoderConfig.load_pretrained:
#     model.encoder.load_from('/content/drive/MyDrive/sunandini/swin_tiny_patch4_window7_224 (1).pth')
# if ASPPConfig.aspp_name == 'swin' and ASPPConfig.load_pretrained:
#     model.aspp.load_from('/content/drive/MyDrive/sunandini/swin_tiny_patch4_window7_224 (1).pth')
# if DecoderConfig.decoder_name == 'swin' and DecoderConfig.load_pretrained and not DecoderConfig.extended_load:
#     model.decoder.load_from('/content/drive/MyDrive/sunandini/swin_tiny_patch4_window7_224 (1).pth')
# if DecoderConfig.decoder_name == 'swin' and DecoderConfig.load_pretrained and DecoderConfig.extended_load:
#     model.decoder.load_from_extended('/content/drive/MyDrive/sunandini/swin_tiny_patch4_window7_224 (1).pth')

### checking the device : cuda or cpu

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device:", device)

device: cuda


# defining the trainloader and validation data loader functions by applying transformations

In [15]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as F

class ImageMaskDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_files = os.listdir(image_folder)
        self.mask_files = os.listdir(mask_folder)
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Open the corresponding image and mask files
        img = Image.open(os.path.join(self.image_folder, self.image_files[idx])).convert("RGB")
        mask = Image.open(os.path.join(self.mask_folder, self.mask_files[idx])).convert("L")
        #print(img.mode)
        # Apply the transformations to the input image and segmentation mask
        if self.transform:
            img = self.transform(img)
            #mask = F.resize(mask, (224, 224))
            mask = self.transform(mask)

        # Normalize the input image
        img = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(img)
        #mask  = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(mask)
        
        return img, mask

# Define the transformations to be applied
transform = transforms.Compose([ transforms.Resize((224, 224)), transforms.ToTensor()])
#transform.transforms[-1].requires_grad = True
# Set the paths to the image and mask folders

train_image_folder = "C:/Users/rajes/OneDrive/Desktop/dataset/data/train/images/"
train_mask_folder ="C:/Users/rajes/OneDrive/Desktop/dataset/data/train/masks/"
val_image_folder =  "C:/Users/rajes/OneDrive/Desktop/dataset/data/val/images/"
val_mask_folder = "C:/Users/rajes/OneDrive/Desktop/dataset/data/val/masks/"
# Create the dataset
train_dataset = ImageMaskDataset(train_image_folder, train_mask_folder, transform=transform)
val_dataset = ImageMaskDataset(val_image_folder, val_mask_folder, transform=transform)
# Create the DataLoader
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Training the model

In [ ]:
model = model.to(device)
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the binary cross-entropy loss function
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001,weight_decay=0.001)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.01)

train_losses = []
train_accs = []
val_losses = []
val_accs = []

# Set the number of epochs
n_epochs = 200

# Check if there are multiple GPUs available
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)
else:
    print("Using single GPU")

best_val_loss = float('inf')
min_epochs=10
for epoch in range(n_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # Compute total number of training examples processed in the current epoch
        total_train += labels.numel()
        # Compute number of correctly predicted training examples in the current batch
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        correct_train += (predicted == labels).sum().item()
        # if i % 10 == 9:  # Print every 10 mini-batches
        #     print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 10))
    # Calculate total training loss over all batches in the current epoch
    train_loss = running_loss / len(train_loader.dataset)
    # Calculate training accuracy over all batches in the current epoch
    train_acc = correct_train / total_train
    print('Epoch %d | Training Loss: %.4f | Training Accuracy: %.2f %%' % (epoch + 1, train_loss, train_acc))
# append current epoch's accuracy and loss values to the respective lists
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    
    # Evaluate the model on the validation set
    model.eval()
    correct_val = 0
    val_loss = 0.0
    total_val = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            outputs = torch.sigmoid(outputs)
            predicted = (outputs > 0.5).float()
            #print("predicted",predicted)
            
            val_loss += loss.item()
            correct_val += (predicted == labels).sum().item()
            total_val += labels.numel()
        val_loss /= len(val_loader)
        # Save the model weights if the validation loss is smaller than the current best validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({'model_state_dict': model.state_dict()}, 'best_model_weights.pth')
        val_acc = correct_val / total_val
        val_losses.append(val_loss)
        val_accs.append(val_acc)
            # check if early stopping criterion is met
        # if epoch > min_epochs and val_loss > best_val_loss:
        #     print('Early stopping!')
        #     break
    print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(epoch+1, n_epochs, val_loss, val_acc))
    # Save the model weights every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({'model_state_dict': model.state_dict()}, 'model_weights_epoch{}.pth'.format(epoch+1))





# Plotting the loss and accuracy curves

In [ ]:
# plot accuracy and loss curves for all epochs so far
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# plot training and validation loss curves
axes[0].plot(train_losses, label='Training Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()


# plot training and validation accuracy curves
axes[1].plot(train_accs, label='Training Accuracy')
axes[1].plot(val_accs, label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy')